In [14]:
import cplex
from cplex.exceptions import CplexError
from docplex.mp.model import *
import random
import matplotlib.pyplot as plt
import docplex.mp.model as Solution
import docplex.cp.parameters as param
import numpy as np
import pandas as pd
import math as math
import csv

In [15]:
data = []
cost = []
t = [] # thời gian di chuyển từ điểm i đến điểm j

# Read the CSV file
with open('distance_matrix.csv', 'r', encoding="utf-8-sig") as file:
    csv_reader = csv.reader(file)

    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Convert the elements in the row from strings to integers
        distance = [float(element) for element in row]
        data.append(distance)
        t.append(distance)
        
    

Lấy các ràng buộc từ khách hàng

In [16]:
data_demands = []
ready_time = []
due_time = []
service_time = []

# Read the customer data from the text file
with open('input.txt', 'r') as file:
    # Skip the header line
    next(file)

    # Process each line in the file
    for line in file:
        # Remove leading/trailing whitespace and split the line into attributes
        attributes = line.strip().split()
        if len(attributes) >= 6:
        # Convert customer number, X coordinate, and Y coordinate to integers
            demand = int(attributes[3])
            redtime = int(attributes[4])
            dutime  =  int(attributes[5])
            sertime = int(attributes[6])

            # Add the dictionary to the data list
            data_demands.append(demand)
            ready_time.append(redtime)
            due_time.append(dutime)
            service_time.append(sertime)

Khai báo các chỉ số cơ bản

In [17]:
# n  = len(data) - 1 #số lượng khách hàng
n = 25 # số lượng khách hàng
Q = 200 #sức chứa của xe
K = 25 #số lượng xe
V = [i for i in range(1,K+1)] # set of vehicle
C = [i for i in range(1,n+1)] #set of client
N = [0] + C #set of node (warehouse + client)
q = {i:data_demands[i] for i in C} #yêu cầu của khách hàng i
s = {i:service_time[i] for i in C} #thời gian phục vụ khách hàng i
a = {i:ready_time[i] for i in C} #thời gian sớm nhất phục vụ khách hàng i
b = {i:due_time[i] for i in C} #thời gian muộn nhất phục vụ khách hàng i
sum(q.values())

540

In [18]:
A = [(i, j) for i in N for j in N if i != j]
c = {(i, j): data[i][j] for i, j in A}

In [19]:
mdl = Model('VRPTW')

In [20]:
x = mdl.binary_var_dict(A, name='x')
w = mdl.continuous_var_dict(C, name='w')
u = mdl.continuous_var_dict(C, ub=Q, name='u') 


In [21]:
mdl.minimize(mdl.sum(c[i, j]*x[i,j] for i, j in A)) #1
mdl.add_constraints(mdl.sum(x[i, j] for j in N if j != i) == 1 for i in C ) #2
mdl.add_constraints(mdl.sum(x[i, j] for i in N if i != j) == 1 for j in C ) #3
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], u[i]+q[j] == u[j]) for i, j in A if i != 0 and j != 0 ) #4
mdl.add_constraints(u[i] >= q[i] for i in C) #5
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], w[i] + s[i] + t[i][j] <= w[j]) for i, j in A if i != 0 and j != 0)
mdl.add_constraints(w[i] <= b[i] for i in C)
mdl.add_constraints(w[i] >= a[i] for i in C)
# mdl.parameters.timelimit(10)
solution = mdl.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 1886.657029 after 0.00 sec. (0.06 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 75 rows and 55 columns.
MIP Presolve modified 490 coefficients.
Aggregator did 490 substitutions.
Reduced MIP has 1195 rows, 1245 columns, and 4030 nonzeros.
Reduced MIP has 595 binaries, 0 generals, 0 SOSs, and 1090 indicators.
Presolve time = 0.02 sec. (4.48 ticks)
Probing time = 0.02 sec. (8.90 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1195 rows, 1245 columns, and 4030 nonzeros.
Reduced MIP has 595 binaries, 0 generals, 0 SOSs, and 1090 indicators.
Presolve time = 0.00 sec. (3.12 ticks)
Probing time = 0.00 sec. (2.83 ticks)
Clique table members: 2549.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.00 sec. (2.35 t

      0     0       99.4118    50      958.2866      Cuts: 79       86   89.63%
      0     0      101.2036    50      958.2866      Cuts: 61      118   89.44%
*     0+    0                          450.7010      101.2036            77.55%
      0     0      101.2036    50      450.7010      Cuts: 48      132   77.55%
*     0+    0                          405.2273      101.2036            75.03%
      0     0      101.2036    50      405.2273      Cuts: 14      141   75.03%
*     0+    0                          326.3552      101.2036            68.99%
Detecting symmetries...
      0     2      101.2036    50      326.3552      101.2036      141   68.99%
Elapsed time = 0.38 sec. (114.00 ticks, tree = 0.02 MB, solutions = 5)
*    11+    7                          321.8190      101.2036            68.55%
*    11+    9                          317.6239      101.2036            68.14%
*  1234+  842                          316.7936      102.2070            67.74%
*  1244+  842            

In [ ]:
solution.display()

solution for: VRPTW
objective: 462.156
status: OPTIMAL_SOLUTION(2)
x_0_2 = 1
x_0_11 = 1
x_0_14 = 1
x_0_23 = 1
x_1_4 = 1
x_2_5 = 1
x_3_1 = 1
x_4_0 = 1
x_5_7 = 1
x_6_8 = 1
x_7_6 = 1
x_8_3 = 1
x_9_10 = 1
x_10_13 = 1
x_11_22 = 1
x_12_15 = 1
x_13_17 = 1
x_14_12 = 1
x_15_16 = 1
x_16_9 = 1
x_17_0 = 1
x_18_25 = 1
x_19_18 = 1
x_20_0 = 1
x_21_19 = 1
x_22_20 = 1
x_23_21 = 1
x_24_0 = 1
x_25_24 = 1
w_1 = 152.000
w_2 = 50.000
w_3 = 139.000
w_4 = 169.071
w_5 = 70.386
w_6 = 104.567
w_7 = 91.567
w_8 = 120.398
w_9 = 114.991
w_10 = 129.991
w_11 = 59.000
w_12 = 64.000
w_13 = 157.820
w_14 = 35.000
w_15 = 81.810
w_16 = 93.810
w_17 = 179.000
w_18 = 117.000
w_19 = 101.615
w_20 = 134.000
w_21 = 77.000
w_22 = 122.000
w_23 = 65.000
w_24 = 178.000
w_25 = 154.000
u_1 = 130.000
u_2 = 30.000
u_3 = 110.000
u_4 = 170.000
u_5 = 50.000
u_6 = 90.000
u_7 = 70.000
u_8 = 100.000
u_9 = 140.000
u_10 = 170.000
u_11 = 40.000
u_12 = 80.000
u_13 = 180.000
u_14 = 60.000
u_15 = 100.000
u_16 = 120.000
u_17 = 200.000
u_18 = 100.000
u

In [ ]:
active_arcs = [a for a in A if x[a].solution_value > 0.9]
active_arcs

[(0, 2),
 (0, 11),
 (0, 14),
 (0, 23),
 (1, 4),
 (2, 5),
 (3, 1),
 (4, 0),
 (5, 7),
 (6, 8),
 (7, 6),
 (8, 3),
 (9, 10),
 (10, 13),
 (11, 22),
 (12, 15),
 (13, 17),
 (14, 12),
 (15, 16),
 (16, 9),
 (17, 0),
 (18, 25),
 (19, 18),
 (20, 0),
 (21, 19),
 (22, 20),
 (23, 21),
 (24, 0),
 (25, 24)]

In [ ]:
def find_all_routes(data_list):
    routes = []
    pre_value = -1
    current_route = []
    while True:
        if len(data_list) == 0:
            break
        for i in data_list:
            if pre_value == 0:
                routes.append(current_route.copy())
                current_route.clear()
                pre_value = -1
                break
            if i[0] == 0 and len(current_route) == 0:
                current_route.extend([i[0], i[1]])
                pre_value = i[1]
                data_list.remove(i)
                continue
            elif i[0] == pre_value:
                current_route.append(i[1])
                pre_value = i[1]
                data_list.remove(i)
                if len(data_list) == 0:
                    routes.append(current_route)
                    break
    print(routes)
    return routes


In [ ]:
def format_route(route):
    return " -> ".join(str(node) for node in route)

In [ ]:
result_routes = find_all_routes(active_arcs)
if result_routes == -1:
    print("No result")
else:
    print("solution for VRPTW")
    print(f"objective: {solution.get_objective_value()}")
    for i, route in enumerate(result_routes, 1):
        print(f"Route {i}: {format_route(route)}")


[[0, 2, 5, 7, 6, 8, 3, 1, 4, 0], [0, 11, 22, 20, 0], [0, 14, 12, 15, 16, 9, 10, 13, 17, 0], [0, 23, 21, 19, 18, 25, 24, 0]]
solution for VRPTW
objective: 462.15594675837383
Route 1: 0 -> 2 -> 5 -> 7 -> 6 -> 8 -> 3 -> 1 -> 4 -> 0
Route 2: 0 -> 11 -> 22 -> 20 -> 0
Route 3: 0 -> 14 -> 12 -> 15 -> 16 -> 9 -> 10 -> 13 -> 17 -> 0
Route 4: 0 -> 23 -> 21 -> 19 -> 18 -> 25 -> 24 -> 0
